In [ ]:
import numpy as np
import mindspore
import mindspore.nn as nn
from mindspore import Parameter, Tensor
from mindspore.ops import operations as P
from mindspore.common.api import ms_function

In [ ]:
def make_batch(sentence, word_dict, n_class, max_len):
    input_batch = []
    target_batch = []

    words = sentence.split()
    for i, word in enumerate(words[:-1]):
        input = [word_dict[n] for n in words[:(i + 1)]]
        input = input + [0] * (max_len - len(input))
        target = word_dict[words[i + 1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch

In [ ]:
class BiLSTM(nn.Cell):
    def __init__(self, n_class, n_hidden, batch_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        self.W = nn.Dense(n_hidden * 2, n_class, has_bias=False)
        self.b = Parameter(Tensor(np.ones([n_class], dtype=np.float32), mindspore.float32))
        self.trans = P.Transpose()
        self.perm = (1, 0, 2)
        self.n_hidden = n_hidden
        self.h = Tensor(np.zeros((1 * 2, batch_size, self.n_hidden)).astype(np.float32))
        self.c = Tensor(np.zeros((1 * 2, batch_size, self.n_hidden)).astype(np.float32))
        
    def construct(self, X):
        input = self.trans(X, self.perm)
        output, (_, _) = self.lstm(input, (self.h, self.c))
        outputs = output[-1]
        model = self.W(outputs) + self.b
        
        return model
        

In [ ]:
n_hidden = 5 # number of hidden units in one cell

sentence = (
    'Lorem ipsum dolor sit amet consectetur adipisicing elit '
    'sed do eiusmod tempor incididunt ut labore et dolore magna '
    'aliqua Ut enim ad minim veniam quis nostrud exercitation'
)

word_dict = {w: i for i, w in enumerate(list(set(sentence.split())))}
number_dict = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word_dict)
max_len = len(sentence.split())
vocab_size = len(word_dict)

In [ ]:
input_batch, target_batch = make_batch(sentence, word_dict, n_class, max_len)
# print(input_batch, target_batch)
input_batch = Tensor(input_batch, mindspore.float32)
target_batch = Tensor(target_batch, mindspore.int32)
print(input_batch.shape, target_batch.shape)

batch_size = len(input_batch)

In [ ]:
model = BiLSTM(n_class, n_hidden, batch_size)

In [ ]:
criterion = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)

In [ ]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

net_with_criterion = nn.WithLossCell(model, criterion)
train_network = nn.TrainOneStepCell(net_with_criterion, optimizer)
train_network.set_train()

In [ ]:
epoch = 10000
for step in range(epoch):
    loss = train_network(input_batch, target_batch)
    if (step + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (step + 1), 'cost = ', '{:.6f}'.format(loss.asnumpy()))

In [ ]:
train_network.set_train(False)
predict = model(input_batch).asnumpy().argmax(axis=1)
print(sentence)
print([number_dict[n.item()] for n in predict.squeeze()])